(curve_similarity_measures)=
# Curve Similarity Measures

## Introduction
Suppose we have a curve representation method, for example some parameterization scheme such as a
spline. We also have a particular shape specified (say the [Stanford Bunny](#stanford_bunny)) and we
want our parameterization to represent that shape as closely as possible. To represent the given
shape one way would be to tune the parameters of the representation scheme iteratively using a
gradient descent optimization approach. This requires us to define an objective function that can
then be optimized over. An appropriate objective for this task would be some measure of
similarity(or dissimilarity) between the target curve and the one traced out by our parameterization
. The objective function can then be maximized(or minimized) to fit the parameters.

The target of this tutorial is to study **curve similarity measures**. We discuss different kinds of
measures and also see their implementations.

:::::{grid} 2

::::{grid-item}
:::{figure} assets/a_basic_spline.svg
:label: a_basic_spline
:width: 100%
:alt: A basic spline drawing to represent a parameterized shape.
Parameterized shape, using a spline.
:::
::::

::::{grid-item}
:::{figure} assets/stanford_bunny.svg
:label: stanford_bunny
:width: 60%
:alt: The Stanford Bunny shown as a target curve.
Target shape, the [Stanford Bunny](https://en.wikipedia.org/wiki/Stanford_bunny).
:::
::::

:::::

## Concrete Problem Setup
First we define the different objects that we deal with:

Shape Parameterization
: We use some parameterization scheme e.g. splines to represent our shapes. We have a set of
parameters $\phi$ that represent our shape. By changing $\phi$ we trace out different curves in the
plane. We will think of $\phi$ as a column vector $[\phi_1, \phi_2, \ldots, \phi_n]^{T}$.

Parameterized Curve
: This is the curve that is traced out by the parameterization scheme. We denote it by $C_p$ and is
obtained by sampling the scheme at different points along the actual curve. It is specified in the
form of a $N_p$ length sequence of $(x, y)$ points. These points are ordered along the curve. We
will specify the points in a matrix in $\mathbb{R}^{N_p \times 2}$ where each row corresponds to a
point $(x_i, y_i)$. We denote the matrix as $X_p$.

Target Curve
: This is the curve that we want our parameterization scheme to represent. We denote it by $C_t$ and
it is specified in the form of a $N_t$ length sequence of $(x, y)$ points. These points are ordered
along the curve. We will specify the points in a matrix in $\mathbb{R}^{N_t \times 2}$ as we did for
the parameterized curve. We denote the matrix as $X_t$.

Loss Function
: A function denoted as $\mathcal{L}(X_t, X_p)$ that measures the degree of dissimilarity between
the target curve and the parameterized curve. It should be differentiable to allow us to find
gradients $\frac{d\mathcal{L}}{d\phi}$ that can then be used to run gradient descent.

**_Goal_**: To tune $\phi$ such that our representation scheme traces out the target curve.

## Similarity Measures
We now discuss the different curve similarity measures. For each measure we describe the exact
mathematical definition, practical considerations, modifications to make them differentiable and
implementations in [PyTorch](https://pytorch.org/).

### Mean Squared Error (MSE)

#### Description
:::{note} Assumption
$N_p = N_t = N$. That is, we sample the parameterized curve at exactly $N_t$ points.
:::

The mean squared error loss function computes the average of the squared distance between the
corresponding points on the two curves. Mathematically,
$$
\mathcal{L} = \frac{1}{N} \sum_{i=1}^{N} \left( d(X_{p}[i], X_{t}[i]) \right)^2
$$
where, $X[i]$ denotes $i${sup}`th` row, i.e. $(x_i, y_i)$ and $d$ is a distance function.

Though the measure is quite naive and not very robust, it is very simple and quick to implement and
is also differentiable without any modifications.

:::{figure} assets/mse_visualization.svg
:width: 70%
:alt: Two curves defined by an equal number of points and an MSE loss between corresponding points.

Mean Squared Error (MSE) between two curves visualized.
:::

#### Implementation

```python
import torch

def mse_loss(X_p, X_t):
    # Calculate the squared Euclidean distances between corresponding rows
    squared_distances = torch.sum((X_p - X_t) ** 2, dim = 1)

    # Calculate mean of the squared distances to get the loss
    loss = torch.mean(squared_distances)

    return loss
```

### Fourier Descriptor Matching

#### Description
The idea behind Fourier descriptor matching is to compute the Fourier coefficients of both the
target and the parameterized curve and then use the difference between them as the loss function.

Concretely, given a curve we can approximate it using a complex Fourier series as follows [^3b1b_FourierSeries_video]:
$$
X(t) = \sum_{n = -\infty}^{\infty} c_n e^{n 2 \pi i t} \quad t \in [0, 1)
$$

In Fourier descriptor matching we use the FFT algorithm to compute a finite number of coefficients
$c_n$ for each of the curves which have themselves been sampled(from $X(t)$) at a finite number of
points given by $X_p$ and $X_t$. Let the coefficients be defined in vectors $F_p$ and $F_t$. The
loss is then computed as the mean squared error of the two coefficient vectors. If $k$ is the total
Fourier coefficients computed in each FFT then the loss is given by:
$$
\mathcal{L} = \frac{1}{k} \sum_{i=1}^{k} \left( d(F_{p}[i], F_{t}[i]) \right)^2
$$

:::{note}
Fourier Descriptor Matching works only for **closed curves**. For open curves, it approximates with
a closed curve.
:::

:::{note}
For the loss to be differentiable we require that the FFT be computed in a way that
allows automatic differentiation to work.
:::

[^3b1b_FourierSeries_video]:
    3Blue1Brown's video on Fourier Series explains the formula really well.
    :::{iframe} https://www.youtube.com/embed/r6sGWTCMz2k?si=ISN2KdknXNlmSr4u
    :::

#### Implementation

```python
import torch
import torch.fft

def fourier_descriptor_matching_loss(X_p, X_t, num_descriptors):
    # Compute Fourier transforms (using FFT)
    fft_p = torch.fft.fft(torch.complex(X_p[..., 0], X_p[..., 1]))
    fft_t = torch.fft.fft(torch.complex(X_t[..., 0], X_t[..., 1]))

    # Select relevant descriptors (low frequencies)
    F_p = fft_p[:num_descriptors]
    F_t = fft_t[:num_descriptors]

    # Calculate MSE loss on magnitudes or complex values
    loss = torch.mean(torch.abs(F_p - F_t)**2)
    return loss
```

The implementation works because the FFT is differentiable in pytorch.

### Hausdorff Distance

#### Description
**_Note_**: Most of the information is taken from the Wikipedia page
[Hausdorff Distance](https://en.wikipedia.org/wiki/Hausdorff_distance).

The Hausdorff distance measures how far two subsets of a metric space are from each other.
Informally, two sets are close in the Hausdorff distance if every point of either set is close to
some point of the other set. The Hausdorff distance is the longest distance someone can be forced to
travel by an adversary who chooses a point in one of the two sets, from where they then must travel
to the other set. In other words, it is the greatest of all the distances from a point in one set to
the closest point in the other set.

Let $(M, d)$ be a metric space. For each pair of non-empty subsets $X \subset M$ and $Y \subset M$,
the Hausdorff distance between $X$ and $Y$ is defined as
$$
d_{\mathrm H}(X,Y) := \max\left\{\,\sup_{x \in X} d(x,Y),\ \sup_{y \in Y} d(X,y) \,\right\}
$$

where $\sup$ represents the supremum operator, $\inf$ the infimum operator, and where
$d(a, B) := \inf_{b \in B} d(a,b)$ quantifies the distance from a point $a \in X$ to the subset $B
\subseteq X$.

:::{figure} assets/Hausdorff_distance.svg
:label: Hausdorff_distance
:width: 55%
:alt: Figure showing two simple closed curves and $\sup d(x, Y)$ and $\sup d(y, X)$.
:::

#### Differentiability
The Hausdorff distance is by itself not differentiable if we implement using minimum and maximum
functions. Therefore we need to work with approximations to it such as:

Soft Hausdorff
: Compute the approximate Hausdorff distance by smoothing the minimum operation to ensure
differentiability. In this approach we use a [smooth minimum function](#smooth_min_max) instead of
the minimum function directly.
    1. Let $P_1$ and $P_2$ be the sets of points on the two curves.
    2. Use a soft-minimum function to approximate the minimum distance between points, such as:
    $$
    d_{\text{soft}}(p, P_2) = -\tau \log \left( \sum_{q \in P_2} \exp \left( -\frac{\|p - q\|_2^2}
    {\tau} \right) \right)
    $$
    3. Compute the smoothed Hausdorff distance:
    $$
    \text{Hausdorff}_{\text{soft}}(P_1, P_2) = \frac{1}{|P_1|} \sum_{p \in P_1}
    d_{\text{soft}}(p, P_2) + \frac{1}{|P_2|} \sum_{q \in P_2} d_{\text{soft}}(q, P_1)
    $$

    :::{dropdown} LogSumExp - Smooth maximum
    :label: smooth_min_max
    The LogSumExp (LSE) function is a smooth maximum – a smooth approximation to the maximum
    function. It is defined as the logarithm of the sum of the exponentials of the arguments:
    $$
    \mathrm{LSE}(x_1, \ldots, x_n) = \log\left( \exp(x_1) + \cdots + \exp(x_n) \right)
    $$

    Writing $\mathbf{x} = (x_1, \ldots, x_n)$ the partial derivatives are:
    $$
    \frac{\partial}{\partial x_i}{\mathrm{LSE}(\mathbf{x})} = 
    \frac{\exp x_i}{\sum_j \exp {x_j}}
    $$
    which means the gradient of LogSumExp is the softmax function.

    Sometimes, a parameter $\tau$ called the temperature parameter is used
    $$
    \mathrm{LSE}(x_1, \ldots, x_n) = \tau \log\left( \exp\left(\frac{x_1}{\tau}\right) + \cdots +
    \exp\left(\frac{x_n}{\tau}\right) \right)
    $$
    $\tau$ controls the sharpness of the approximation. As $\tau$ approaches 0, the softmin
    approaches the true minimum.[^temperature_param]

    Also, note that $\min \left( x, y \right) = -\max \left(-x, -y \right)$. We can use this to get
    the smooth minimum function using the LogSumExp.
    :::


[^temperature_param]: $\tau$ is called the temperature parameter because as $\tau \to \infty$ too
    much smoothing happens and all values tend to the same, i.e. at very high temperature behavior
    is random. If we cool down $\tau$ then the it tends to the true $\max$ function.

#### Implementation

```python
import torch

def hausdorff_soft_loss(X_p, X_t, tau = 1.0):
    # Compute pairwise squared distances
    dist_matrix = torch.cdist(P1, P2) ** 2  # Shape: (N1, N2)
    
    # Compute soft-minimum distances
    d_soft_p1_to_p2 = -tau * torch.logsumexp(-dist_matrix / tau, dim = 1)  # Shape: (N1,)
    d_soft_p2_to_p1 = -tau * torch.logsumexp(-dist_matrix.T / tau, dim = 1)  # Shape: (N2,)

    # Compute the soft Hausdorff loss
    hausdorff_soft = d_soft_p1_to_p2.mean() + d_soft_p2_to_p1.mean()
    
    return hausdorff_soft.item()
```